In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils
from PIL import Image
import torch
import tensorflow as tf
import tensorflow_datasets as tfds
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#Downloal dataset from tensorflow website

# this block of code is retrived from https://stackoverflow.com/questions/44429199/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader

class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        if self.transform:
            x = Image.fromarray(self.data[index].astype(np.uint8).transpose(1,2,0))
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

# download dataset as numpy
dp_tf = tfds.load(name="deep_weeds", split = ["train"])
dp_numpy = tfds.as_numpy(dp_tf)

In [ ]:
# convert data to numpy array
train_images = []
train_labels = []

for image in dp_numpy[0]:
    train_images.append(image['image'])
    train_labels.append(image['label'])

In [ ]:
# convert numpy array to pytorch dataset
train_images = np.array(train_images)
train_images = np.transpose(train_images, (0, 3, 1, 2))
d_length = len(train_images)
train_labels = np.array(train_labels)

In [ ]:
#just some usual process of preparing dataset with pytorch
transform = transforms.Compose([transforms.RandomCrop(256), transforms.RandomHorizontalFlip(),
                                transforms.ToTensor()])
test_transform = transforms.Compose(
[transforms.ToTensor()])

dset = MyDataset(train_images, train_labels, transform=transform)
train_set, test_set = torch.utils.data.random_split(dset, lengths=[int(d_length * 0.95), int(d_length - int(d_length * 0.95))],
                                                    generator=torch.Generator().manual_seed(42))

trainloader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

In [ ]:
#decide whether to use pretrained efficientnet
pre_trained = False
if pre_trained:
    net = EfficientNet.from_pretrained('efficientnet-b2') #here we should use b2 but not not b0
else:
    net = EfficientNet.from_name("efficientnet-b2")

#get the connected gpu in use
net = net.cuda()
net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))

In [ ]:
#train

num_epochs = 500
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

loss_list = []
test_acc_list = []
lrSchduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    test_loss = 0.0
    net.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        # print statistics
        running_loss += loss.data.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Normalizing the loss by the total number of train batches
    running_loss /= len(trainloader)

    # Calculate training accuracy of the existing model

    train_accuracy = correct / total
    loss_list.append(running_loss)

    test_correct = 0
    test_total = 0

    net.eval()
    with torch.no_grad():
        for data in testloader:
            test_images, test_labels = data
            test_inputs = test_images.cuda()
            test_labels = test_labels.cuda()
            test_outputs = net(test_inputs)
            test_loss = criterion(test_outputs, test_labels)
            _, predicted = torch.max(test_outputs.data, 1)
            test_total += test_labels.size(0)
            test_correct += (predicted == test_labels).sum().item()
    test_acc = test_correct / test_total
    test_acc_list.append(test_acc)

    print("Iteration: {0} | Loss: {1} | Training accuracy: {2} | test loss: {3} | test_acc: {4}".format(epoch+1,
                                                                                              running_loss,
                                                                                              train_accuracy, test_loss,
                                                                                                   test_acc
                                                                                            ))
    lrSchduler.step()
    print('==> Finished Training ...')

In [ ]:
best_test_acc = max(test_acc_list)
index = test_acc_list.index(max(test_acc_list)) + 1
print("Best test acc during training: ", best_test_acc, " epoch: ", index)

print("done")